### Import Libraries

In [1]:
import numpy as np
import pandas as pd
print(pd.__version__)
pd.options.display.max_colwidth = 500

0.20.3


## Import CSV's into pandas dataframes

In [27]:
df_submissions = pd.read_csv("reddit_submissions_wUTC_2018-05-10_1022.csv",
skipinitialspace=True,
header = 0,
sep = ',',
index_col=0,
encoding='latin1').fillna('')
df_submissions['post'] = 1
print(df_submissions.shape)
print(df_submissions.columns)

df_comments = pd.read_csv("reddit_comments_wUTC2018-05-10_1022.csv",
skipinitialspace=True,
header = 0,
sep = ',',
index_col=0,
encoding='latin1').fillna('')
df_comments['post'] = 1
print(df_comments.shape)
print(df_comments.columns)

(1362, 26)
Index(['id', 'subreddit_id', 'title', 'score', 'author', 'author_flair_text',
       'likes', 'archived', 'is_video', 'name', 'over_18', 'pinned',
       'quarantine', 'selftext', 'send_replies', 'num_comments',
       'num_crossposts', 'distinguished', 'created', 'created_utc',
       'comment_limit', 'approved_by', 'ups', 'downs', 'gilded', 'post'],
      dtype='object')
(6704, 20)
Index(['comment_id', '_submission', 'subreddit_id', 'score', 'author',
       'author_flair_text', 'archived', 'name', 'body', 'send_replies',
       'distinguished', 'created', 'created_utc', 'approved_by', 'link_id',
       'parent_id', 'ups', 'downs', 'gilded', 'post'],
      dtype='object')


## Renaming Columns

In [3]:
df_comments = df_comments.rename(columns={'_submission':'id',
                                         'body':'selftext'})

## Merge Dataframes 

In [4]:
df_all = df_submissions.append(df_comments)
df_all['comment'] = np.where(df_all['comment_id'].isnull(),0,1)
df_all['submission'] = np.where(df_all['comment']==0,1,0)
print(df_all.shape)
df_all.columns

(8066, 31)


Index(['approved_by', 'archived', 'author', 'author_flair_text', 'comment_id',
       'comment_limit', 'created', 'created_utc', 'distinguished', 'downs',
       'gilded', 'id', 'is_video', 'likes', 'link_id', 'name', 'num_comments',
       'num_crossposts', 'over_18', 'parent_id', 'pinned', 'post',
       'quarantine', 'score', 'selftext', 'send_replies', 'subreddit_id',
       'title', 'ups', 'comment', 'submission'],
      dtype='object')

## Format variables related to time

The date of Reddit's posts are stored in seconds <br>
* the initial date is '1970-01-01 00:00:00' <br>
* The unit argument stands for the unit of time measure : 's'
* Split the date and the time in separeted variables
* The variable 'created' is reffering to the Timestamp from ? ... We can convert it to UTC by removing 8 hours

In [5]:
start = 0
start = pd.to_datetime(start, unit='s')
print(start)

1970-01-01 00:00:00


### Compare created & created_UTC

In [15]:
df_all['datetime_utc'] = pd.to_datetime(df_all['created_utc'], unit='s')
df_all['datetime_utc'] = pd.DatetimeIndex(df_all['datetime_utc'])
print(df_all['datetime_utc'][0:5])

2   2018-05-03 16:44:18
4   2018-05-03 17:02:20
5   2018-05-03 11:51:18
6   2018-05-02 19:40:32
9   2018-05-02 19:33:26
Name: datetime_utc, dtype: datetime64[ns]


### Format Date & Time

In [17]:
df_all['datetime'] = pd.to_datetime(df_all['created'], unit='s')
df_all['datetime'] = pd.DatetimeIndex(df_all['datetime'])
print(df_all['datetime'][0:5])

2   2018-05-04 00:44:18
4   2018-05-04 01:02:20
5   2018-05-03 19:51:18
6   2018-05-03 03:40:32
9   2018-05-03 03:33:26
Name: datetime, dtype: datetime64[ns]


In [18]:
temp = pd.to_datetime(df_all['created'], unit='s')
temp = pd.DatetimeIndex(temp)
print(temp[0:2])

DatetimeIndex(['2018-05-04 00:44:18', '2018-05-04 01:02:20'], dtype='datetime64[ns]', name='created', freq=None)


In [19]:
df_all['date'] = temp.date
df_all['time'] = temp.time
print(df_all['date'][0:2])
print(df_all['time'][0:2])

2    2018-05-04
4    2018-05-04
Name: date, dtype: object
2    00:44:18
4    01:02:20
Name: time, dtype: object


## Time series Analysis

down vote
accepted
B       business day frequency
C       custom business day frequency (experimental)
D       calendar day frequency
W       weekly frequency
M       month end frequency
SM      semi-month end frequency (15th and end of month)
BM      business month end frequency
CBM     custom business month end frequency
MS      month start frequency
SMS     semi-month start frequency (1st and 15th)
BMS     business month start frequency
CBMS    custom business month start frequency
Q       quarter end frequency
BQ      business quarter endfrequency
QS      quarter start frequency
BQS     business quarter start frequency
A       year end frequency
BA      business year end frequency
AS      year start frequency
BAS     business year start frequency
BH      business hour frequency
H       hourly frequency
T       minutely frequency
S       secondly frequency
L       milliseonds
U       microseconds
N       nanoseconds

In [20]:
from pandas import Series, DataFrame, Panel, Grouper

In [21]:
df_all = df_all.set_index('datetime')

In [22]:
sub_permonth = df_all['submission'].resample("M").sum()
print(sub_permonth)

comment_permonth = df_all['comment'].resample("M").sum()
print(comment_permonth)

datetime
2017-11-30     58
2017-12-31    185
2018-01-31    231
2018-02-28    260
2018-03-31    278
2018-04-30    293
2018-05-31     57
Freq: M, Name: submission, dtype: int64
datetime
2017-11-30     229
2017-12-31     890
2018-01-31    1100
2018-02-28    1107
2018-03-31    1402
2018-04-30    1590
2018-05-31     386
Freq: M, Name: comment, dtype: int64


In [23]:
sub_permonth = df_all['submission'].resample("W").sum()
print(sub_permonth)

comment_permonth = df_all['comment'].resample("W").sum()
print(comment_permonth)

datetime
2017-11-26    19
2017-12-03    57
2017-12-10    38
2017-12-17    45
2017-12-24    50
2017-12-31    34
2018-01-07    52
2018-01-14    59
2018-01-21    54
2018-01-28    38
2018-02-04    63
2018-02-11    70
2018-02-18    51
2018-02-25    60
2018-03-04    85
2018-03-11    70
2018-03-18    39
2018-03-25    80
2018-04-01    54
2018-04-08    51
2018-04-15    79
2018-04-22    69
2018-04-29    77
2018-05-06    53
2018-05-13    15
Freq: W-SUN, Name: submission, dtype: int64
datetime
2017-11-26     74
2017-12-03    225
2017-12-10    198
2017-12-17    217
2017-12-24    274
2017-12-31    131
2018-01-07    208
2018-01-14    260
2018-01-21    291
2018-01-28    220
2018-02-04    257
2018-02-11    256
2018-02-18    221
2018-02-25    299
2018-03-04    373
2018-03-11    328
2018-03-18    254
2018-03-25    424
2018-04-01    243
2018-04-08    336
2018-04-15    448
2018-04-22    419
2018-04-29    320
2018-05-06    326
2018-05-13    102
Freq: W-SUN, Name: comment, dtype: int64


In [26]:
sub_permonth = df_all['submission'].resample("D").sum()
print('submissions : {}'.format(sub_permonth))

comment_permonth = df_all['comment'].resample("D").sum()
print('comments : {}'.format(comment_permonth))

submissions : datetime
2017-11-23     7
2017-11-24     5
2017-11-25     4
2017-11-26     3
2017-11-27     8
2017-11-28    20
2017-11-29     7
2017-11-30     4
2017-12-01     6
2017-12-02     3
2017-12-03     9
2017-12-04     2
2017-12-05     6
2017-12-06     4
2017-12-07     9
2017-12-08    12
2017-12-09     4
2017-12-10     1
2017-12-11     3
2017-12-12    14
2017-12-13     8
2017-12-14     8
2017-12-15    10
2017-12-16     2
2017-12-17     0
2017-12-18     9
2017-12-19    10
2017-12-20     4
2017-12-21     9
2017-12-22     8
              ..
2018-04-11    11
2018-04-12    19
2018-04-13    15
2018-04-14     5
2018-04-15    11
2018-04-16    11
2018-04-17    20
2018-04-18    12
2018-04-19    11
2018-04-20     6
2018-04-21     4
2018-04-22     5
2018-04-23     4
2018-04-24    25
2018-04-25    12
2018-04-26    16
2018-04-27     6
2018-04-28     8
2018-04-29     6
2018-04-30    11
2018-05-01    13
2018-05-02    11
2018-05-03     6
2018-05-04     8
2018-05-05     3
2018-05-06     1
2018-05-